In [29]:
import os
import json
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from typing import Dict, List, Any

In [30]:
class MobileAppSecurityFramework:
    def __init__(self, config_path: str = 'config.json'):
        """
        Initialize the Mobile App Security Testing Framework

        :param config_path: Path to configuration file
        """
        # Load configuration
        with open(config_path, 'r') as f:
            self.config = json.load(f)

        # Initialize components
        self.vulnerability_model = None
        self.dataset = None
        self.preprocessor = None

In [31]:
    def load_dataset(self, dataset_path: str):
        """
        Load and preprocess the mobile app security dataset

        :param dataset_path: Path to dataset
        """
        self.dataset = pd.read_csv(dataset_path)

        # Preprocessing
        self.preprocessor = StandardScaler()

        # Separate features and labels
        X = self.dataset.drop('vulnerability_label', axis=1)
        y = self.dataset['vulnerability_label']

        # Split the data
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )

        # Scale features
        self.X_train_scaled = self.preprocessor.fit_transform(self.X_train)
        self.X_test_scaled = self.preprocessor.transform(self.X_test)

In [32]:
    def build_ml_model(self):
        """
        Build and compile the vulnerability detection neural network
        """
        self.vulnerability_model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu', input_shape=(self.X_train_scaled.shape[1],)),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])

        self.vulnerability_model.compile(
            optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy']
        )

In [33]:
    def train_model(self, epochs: int = 50, batch_size: int = 32):
        """
        Train the vulnerability detection model

        :param epochs: Number of training epochs
        :param batch_size: Training batch size
        """
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )

        history = self.vulnerability_model.fit(
            self.X_train_scaled,
            self.y_train,
            validation_split=0.2,
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[early_stopping]
        )

        return history

In [34]:
    def evaluate_model(self):
        """
        Evaluate the model's performance

        :return: Comprehensive performance metrics
        """
        # Predict on test data
        y_pred = (self.vulnerability_model.predict(self.X_test_scaled) > 0.5).astype(int)

        # Generate classification report
        report = classification_report(
            self.y_test,
            y_pred,
            target_names=['No Vulnerability', 'Vulnerability']
        )

        # Confusion Matrix
        cm = confusion_matrix(self.y_test, y_pred)

        return {
            'classification_report': report,
            'confusion_matrix': cm
        }

In [35]:
    def detect_vulnerabilities(self, app_features: Dict[str, Any]) -> Dict[str, float]:
        """
        Detect vulnerabilities in a mobile application

        :param app_features: Dictionary of application features
        :return: Vulnerability scores and types
        """
        # Convert input to scaled features
        features_array = self.preprocessor.transform(
            pd.DataFrame([app_features])
        )

        # Predict vulnerability probability
        vulnerability_prob = self.vulnerability_model.predict(features_array)[0][0]

        vulnerability_types = {
            'insecure_storage': vulnerability_prob * 0.4,
            'weak_encryption': vulnerability_prob * 0.3,
            'api_misuse': vulnerability_prob * 0.3
        }

        return {
            'total_vulnerability_score': vulnerability_prob,
            'vulnerability_breakdown': vulnerability_types
        }

In [36]:
    def generate_security_report(self, results: Dict[str, Any]) -> str:
        """
        Generate a detailed security report

        :param results: Vulnerability detection results
        :return: Formatted security report
        """
        report = "Mobile App Security Assessment Report\n"
        report += "=" * 40 + "\n\n"

        report += f"Total Vulnerability Score: {results['total_vulnerability_score']:.2%}\n\n"

        report += "Vulnerability Breakdown:\n"
        for vuln_type, score in results['vulnerability_breakdown'].items():
            report += f"- {vuln_type.replace('_', ' ').title()}: {score:.2%}\n"

        return report

In [37]:
# Main execution script
def main():
    # Initialize framework
    security_framework = MobileAppSecurityFramework()

    # Load dataset
    security_framework.load_dataset('mobile_app_vulnerabilities.csv')

    # Build ML model
    security_framework.build_ml_model()

    # Train model
    training_history = security_framework.train_model()

    # Evaluate model
    model_performance = security_framework.evaluate_model()

    # Example app vulnerability detection
    sample_app_features = {
        'storage_encryption_level': 0.3,
        'api_security_score': 0.2,
        'data_transmission_security': 0.4
    }

    vulnerability_results = security_framework.detect_vulnerabilities(sample_app_features)

    # Generate security report
    security_report = security_framework.generate_security_report(vulnerability_results)
    print(security_report)


if __name__ == "__main__":
    main()


AttributeError: 'MobileAppSecurityFramework' object has no attribute 'load_dataset'